In [11]:
import pandas as pd
import numpy as np

#for all your http queries
import requests

#to deal with time stuff
import datetime

In [12]:
#you can get this file by opening "network" in developer tools
#in your Robinhood "history" tab and "copy response" on "orders/"
#there are 3 "orders/", one is your stock trade history, one is
#your option trade history, one is unknown because it's empty

#then go to json-csv.com, past the copied json and download the csv
#name it what you want and open it here

directory = r'C:\Users\geeze\Documents\Finances\stocks'
file = '\\20181211_rh_hist.csv'

raw_hist = pd.read_csv(directory + file)

In [13]:
#there is a persistent row that doesn't have any useful information, just a single non NaN value somewhere in it
#I want to drop it. When asked what type it is, it says "float", a real entry is "str", so drop the non-str.
idx_to_keep = raw_hist['results__updated_at'].apply(type) == str

raw_hist = raw_hist.loc[idx_to_keep, :].reset_index(drop=True)

In [14]:
def get_date_and_time (series_of_times):
    #expand splits the string at the delimiter and creates 2 new dataframe columns with the results
    #the 'T' character separates the date and the time parts of these entries
    date_and_time = series_of_times.str.split('T', expand=True)
    
    #the date is in column 0
    date = date_and_time[0]
    #the time is in 1, but needs splitting to get rid of decimal seconds that follow a '.'
    time = date_and_time[1].str.split('.', expand=True)[0]
    
    return date, time

In [15]:
#Start building the dataframe with information you actually want

place_date, place_time = get_date_and_time(raw_hist['results__updated_at'])
exec_date, exec_time = get_date_and_time(raw_hist['results__executions__timestamp'])

            #first add the times in
raw_hist = (raw_hist.assign(place_date = place_date, place_time = place_time, exec_date = exec_date, exec_time = exec_time)
            #then rename all the columns you'll want to keep
            .rename(columns={'results__state' : 'state',
                             'results__time_in_force' : 'time in force',
                             'results__type' : 'order type',
                             'results__price' : 'entered price',
                             'results__executions__price' : 'executed price',
                             'results__average_price' : 'avg price',
                             'results__quantity' : 'num shares',
                             'results__side' : 'side',
                             'results__fees' : 'fees',
                             'place_date' : 'date placed',
                             'place_time' : 'time placed',
                             'exec_date' : 'date executed',
                             'exec_time' : 'time executed'
                            })
            #get the ticker symbols by getting the json text from the api.robinhood.com/instruments url
            .assign(symbol = [requests.get(url).json()['symbol'] for url in raw_hist['results__instrument']])
            #get rid of misc columns that dont contribute
            .drop(labels=['previous', 'next'], axis='columns')
           )

In [16]:
#all the raw column names have "results__" in them, remove these and get just the mature df you will use
history = raw_hist[[col for col in raw_hist.columns if 'results__' not in col]]

In [18]:
#save result

now = datetime.datetime.now()

history.to_csv('{}_tidy_rh_hist.csv'.format(now.strftime("%Y%m%d")), index=False)

### next step is to read in existing history sheet, compare what you have just processed to what you already have and see if there are mismatches or something. If robinhood always gives you everything every time, then I guess it's no big deal and you can just save more and more recent files

### I want to be able to automatically log in to robinhood and then download this orders json so I don't have to manually do it.

In [195]:
test = requests.get('https://robinhood.com/login', auth=('drreedmccardell@gmail.com', '01011991Rm'))

In [196]:
test.content

b'\n\n\n\n\n\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <link\n      href="https://d2ue93q3u507c2.cloudfront.net/assets/favicon.ico"\n      rel="shortcut icon"\n    >\n\n    <link rel="stylesheet" href="https://d2ue93q3u507c2.cloudfront.net/assets/generated_assets/App.3baafbb38f75ef7ecfab.css" type="text/css" media="all"/>\n  </head>\n\n  <body class="theme-open-up">\n    \n      <div id="react_root">\n        <div id="application_loading"></div>\n      </div>\n      <script nonce="j9ksjo+7c+3GYfcBHi9SEB/WhjJnfTUw">\n        setTimeout(function() {\n          const loadingBlock = document.getElementById("application_loading")\n          if (loadingBlock) {\n            loadingBlock.innerHTML = \'<div style="height: 100vh; display: flex; align-items: center; justify-content: center;">\'+\n              \'<?xml version="1.0" encoding="UTF-8" standalone="no"?><svg width="21px" height="28px" viewBox="0 0 21 28" version="1.1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www